In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import re

In [95]:
# Set up the driver
path = "C:\Program Files\Google\Chrome\chromedriver.exe"
service = Service(executable_path=path)
driver_daraz = webdriver.Chrome(service=service)
driver_dealayo = webdriver.Chrome(service=service)
driver_daraz.get("https://www.daraz.com.np/?spm=a2a0e.shipping.header.dhome.178a6af7ssboMK")
driver_dealayo.get("https://www.dealayo.com/")

driver_daraz.maximize_window()
driver_dealayo.maximize_window()


In [216]:
# Search for the item in daraz
search_box_daraz = driver_daraz.find_element(By.CSS_SELECTOR, "#q")
value = input("Enter the product name: ")
search_box_daraz.send_keys(value)
search_box_daraz.send_keys(Keys.RETURN)

# Search for the item in dealayo
search_box_dealayo = driver_dealayo.find_element(By.CSS_SELECTOR, "#search")
search_box_dealayo.send_keys(value)
search_box_dealayo.send_keys(Keys.RETURN)

# Wait for the page to load
time.sleep(2)

words  =value.split()
last_word = words[-1]
index = words.index(last_word)

Enter the product name: oppo a16


In [217]:
product_links_daraz = []
mob_data_list_daraz = []

# Get the product links
product_elements_daraz = driver_daraz.find_elements(By.XPATH, '//div[@class="mainPic--ehOdr"]/a[@age="0"]')
for product_element_daraz in product_elements_daraz:
    product_links_daraz.append(product_element_daraz.get_attribute("href"))

# Get the data for 8 products from the product link 
for product_link_daraz in product_links_daraz[:8]:
    driver_daraz.get(product_link_daraz)
    time.sleep(2)
    
    # Check if the value of the specified element is "Mobiles"
    breadcrumb_element_daraz = driver_daraz.find_element(By.CSS_SELECTOR, "#J_breadcrumb > li:nth-child(2) > span > a > span")
    breadcrumb_value_daraz = breadcrumb_element_daraz.text
    
    if breadcrumb_value_daraz != "Mobiles":
        break
    
    try:
        mob_data_dict_daraz = {}

        # Get the name
        try:
            mob_detail_element_daraz = driver_daraz.find_element(By.CSS_SELECTOR, '#module_product_title_1 > div > div > span')
            mob_detail_daraz = mob_detail_element_daraz.text            
            mob_detail_daraz = mob_detail_daraz.lower()
            mob_name = mob_detail_daraz
            mob_detail_daraz = re.split(r' [|,\-(]', mob_detail_daraz, 1)[0]
            mob_detail_daraz = mob_detail_daraz.replace("apple ", "")
            mob_data_dict_daraz["Name"] = mob_detail_daraz

        except:continue

        # Get the price
        try:
            price_daraz = driver_daraz.find_element(By.CSS_SELECTOR, value='#module_product_price_1 > div > div > span')
            mob_data_dict_daraz["Price"] = price_daraz.text
        except:continue
            
        # Get the storage
        try:
            storage_daraz = driver_daraz.find_element(By.CSS_SELECTOR, value='#module_sku-select > div > div:nth-child(3) > div > div > div.sku-prop-content-header > span')
            mob_data_dict_daraz["Storage"] = storage_daraz.text
        except:
            matches =re.findall(r'\d+\s*gb', mob_name)
            Storage_gb = matches[1]
            mob_data_dict_daraz["Storage"] =Storage_gb.upper()

        # Get the RAM
        try:
            ram_daraz = driver_daraz.find_element(By.CSS_SELECTOR, value='#module_sku-select > div > div:nth-child(2) > div > div > div.sku-prop-content-header > span')
            mob_data_dict_daraz["RAM"] = ram_daraz.text
        except:
            matches =re.findall(r'\d+\s*gb', mob_name)
            Ram_gb = matches[0]
            mob_data_dict_daraz["RAM"] =Ram_gb.upper()
        
        mob_data_dict_daraz["URL"] = product_link_daraz

        mob_data_list_daraz.append(mob_data_dict_daraz)
        
    except:
        continue


In [218]:
# show all the searched item
for data_daraz in mob_data_list_daraz:
    words = (data_daraz["Name"]).split()
    tot_words = ' '.join(words[:index+1])
    if tot_words == value:
        print("Name:", tot_words)
        print("Price:", data_daraz["Price"])
        print("Storage:", data_daraz["Storage"])
        print("RAM:", data_daraz["RAM"])
        print("URL:", data_daraz["URL"])
        print()


Name: oppo a16
Price: Rs. 19,500
Storage: 64GB
RAM: 4GB
URL: https://www.daraz.com.np/products/oppo-a16-4gb64gb-3d-curved-edge-body-13mp-triple-ai-camera-battery-5000mah-hd-eye-care-display-i125154998-s1033975733.html?search=1

Name: oppo a16
Price: Rs. 20,000
Storage: 64GB
RAM: 4GB
URL: https://www.daraz.com.np/products/oppo-a16-4gb-ram-64gb-rom-5000mah-battery-software-android-11-with-coloros-111-i114995485-s1031465843.html?search=1



In [43]:
# show searhed item with user ram and storage
device_found = False 
user_ram = input("Enter the RAM: ")
user_storage = input("Enter the storage: ")
for data_daraz in mob_data_list_daraz:
    words = (data_daraz["Name"]).split()
    tot_words = ' '.join(words[:index+1])
    if tot_words == value:
        if (data_daraz["RAM"] == user_ram + "GB" or data_daraz["RAM"] == user_ram + " GB") and (data_daraz["Storage"] == user_storage + "GB" or data_daraz["Storage"] == user_storage + " GB"):
            print("Name:", tot_words)
            print("Price:", data_daraz["Price"])
            print("Storage:", data_daraz["Storage"])
            print("RAM:", data_daraz["RAM"])
            print("URL:", data_daraz["URL"])
            device_found = True
if not device_found:
    print(f"{value} is not available in {user_ram}GB RAM with {user_storage}GB storage")

Enter the RAM: 8
Enter the storage: 128
Name: oneplus 8t
Price: Rs. 78,999
Storage: 128GB
RAM: 8GB
URL: https://www.daraz.com.np/products/oneplus-8t-ke2003-ram-8gb-rom-128gb-64mp-quad-camera-i105032170-s1026638529.html?search=1


In [219]:
product_links_dealayo = []
mob_data_list_dealayo = []

try:
    sort_button_dealayo = driver_dealayo.find_element(By.CSS_SELECTOR, 'body > div.wrapper > div > div.main-container.col2-left-layout > div > div.container.col-main-containter > div > div.col-sm-18.col-sm-push-6.col-main > div > div.category-products > div.toolbar > div.sorter > div > select > option:nth-child(3)')
    sort_button_dealayo.click()

    sort_button_dealayo = driver_dealayo.find_element(By.CSS_SELECTOR, 'body > div.wrapper > div > div.main-container.col2-left-layout > div > div.container.col-main-containter > div > div.col-sm-18.col-sm-push-6.col-main > div > div.category-products > div.toolbar > div.sorter > div > a')
    sort_button_dealayo.click()

    # Wait for the page to reload
    time.sleep(2)
    
    # Get the link of each product on the page
    product_elements_dealayo = driver_dealayo.find_elements(By.XPATH, '//div/a[@class="product-image no-alt-img"]')
    for product_element_dealayo in product_elements_dealayo:
        product_links_dealayo.append(product_element_dealayo.get_attribute('href'))
    
    # Iterate through the links of every product
    for product_link_dealayo in product_links_dealayo:
        driver_dealayo.get(product_link_dealayo)
        time.sleep(2)
        
        try:
            mob_data_dict_dealayo = {}

            try:
                name_dealayo_element = driver_dealayo.find_element(By.CSS_SELECTOR, value='#product_addtocart_form > div.row > div > div > div.col-sm-19 > div > div.col-sm-10.col-sm-push-1.product-view-info > div > div.product-shop-wrap.product-shop-title > div.product-name > h1')
                name_dealayo = name_dealayo_element.text
                name_dealayo = re.split(r' [|,\-(]', name_dealayo, 1)[0]
                mob_data_dict_dealayo["Name"] = name_dealayo
            except:
                pass

            try:
                price_element = driver_dealayo.find_element(By.XPATH , '//span[@class="regular-price"]')
                mob_data_dict_dealayo["Price"] = price_element.text
            except:
                pass

            try:
                storage_element = driver_dealayo.find_element(By.XPATH , '//*[@id="product-attribute-specs-table-4"]/tbody/tr[5]/td')
                mob_data_dict_dealayo["Storage"] = storage_element.text
            except:
                pass

            try:
                ram_element = driver_dealayo.find_element(By.XPATH , '//*[@id="product-attribute-specs-table-4"]/tbody/tr[4]/td')
                mob_data_dict_dealayo["RAM"] = ram_element.text
            except:
                pass

            mob_data_dict_dealayo["URL"] = product_link_dealayo
            mob_data_list_dealayo.append(mob_data_dict_dealayo)

        except:
            break

except NoSuchElementException:
    mob_data_dict_dealayo = {}
    try:
        name_dealayo_element = driver_dealayo.find_element(By.CSS_SELECTOR, value='#product_addtocart_form > div.row > div > div > div.col-sm-19 > div > div.col-sm-10.col-sm-push-1.product-view-info > div > div.product-shop-wrap.product-shop-title > div.product-name > h1')
        name_dealayo = name_dealayo_element.text
        name_dealayo = re.split(r' [|,\-(]', name_dealayo, 1)[0]
        mob_data_dict_dealayo["Name"] = name_dealayo
    except:
        pass

    try:
        ram_element = driver_dealayo.find_element(By.XPATH , '//*[@id="product-attribute-specs-table-4"]/tbody/tr[4]/td')
        mob_data_dict_dealayo["RAM"] = ram_element.text
    except:
        pass

    try:
        storage_element = driver_dealayo.find_element(By.XPATH , '//*[@id="product-attribute-specs-table-4"]/tbody/tr[5]/td')
        mob_data_dict_dealayo["Storage"] = storage_element.text
    except:
        pass

    try:
        price_element = driver_dealayo.find_element(By.XPATH , '//span[@class="regular-price"]')
        mob_data_dict_dealayo["Price"] = price_element.text
    except:
        pass

    mob_data_dict_dealayo["URL"] = driver_dealayo.current_url
    mob_data_list_dealayo.append(mob_data_dict_dealayo)



In [220]:
# searched items
for deal in mob_data_list_dealayo:
    print("Name:", deal["Name"])
    print("Price:", deal["Price"])
    print("Storage:", deal["Storage"])
    print("RAM:", deal["RAM"])
    print("URL:", deal["URL"])
    print()
    

Name: Su-vastika 165ah


KeyError: 'Price'

In [212]:
#deal ayo ko searched item with user ram and storage as input
device_found = False
user_ram = input("Enter the RAM: ")
user_storage = input("Enter the storage: ")

for deal in mob_data_list_dealayo:
    words = (deal["Name"]).split()
    tot_words = ' '.join(words[:index+1])
    if tot_words.lower() == value:
        if (deal["RAM"] == user_ram + "GB" or deal["RAM"] == user_ram + " GB") and (deal["Storage"] == user_storage + "GB" or deal["Storage"] == user_storage + " GB"):
            print("Name:", deal["Name"])
            print("Price:", deal["Price"])
            print("Storage:", deal["Storage"])
            print("RAM:", deal["RAM"])
            print("URL:", deal["URL"])
            print()
            device_found = True
            
if not device_found:
    print(f"{deal['Name']} is not available in {user_ram}GB RAM with {user_storage}GB storage")
   


Enter the RAM: 12
Enter the storage: 123
Samsung Galaxy S23 Ultra 5G is not available in 12GB RAM with 123GB storage
